In [1]:
import numpy as np
import statistics
from skimage.io import imread, imsave
from skimage import feature
from scipy import ndimage as ndi
from sklearn.cluster import KMeans
from skimage import img_as_ubyte
from skimage import (filters,
                     morphology,
                     exposure,
                     measure)

from commons import (crop_image_box, 
                     binarize_image,
                     plot_grid_images, 
                     build_volume_from_directory,
                     crop_image_reduce_errors,
                     rule_of_three)

In [2]:
def apply_kmeans(img, k_clusters=3):
    return KMeans(random_state=1,
                  n_clusters=k_clusters,
                  init='k-means++'
                  ).fit(img.reshape((-1, 1))).labels_.reshape(img.shape)


def find_center_mask(image_bin):
    """
        Retona o centro da imagem
    """

    props, *_ = measure.regionprops(
        measure.label(image_bin)
    )

    x, y = props.centroid

    return int(x), int(y)


def find_perfect_cluster(mask):
    """
        Função especial para o kmeans
    """
    max_area = 0
    max_area_index = 0

    for index, props in enumerate(measure.regionprops(mask)):
        if props.area >= max_area:
            max_area = props.area
            max_area_index = index + 1
    return max_area_index


def find_bighest_cluster_area(clusters):
    """
        Essa função deve receber uma imagem segmentada (Clusters)
        Retorna a área do maior cluster
    """
    regions = measure.regionprops(clusters)

    def area(item): return item.area

    return max(map(area, regions))


def find_best_larger_cluster(image_mask):

    clusters = image_mask.copy()

    if statistics.mode(clusters.flatten()):
        clusters = np.invert(clusters)

    clusters_labels = measure.label(clusters, background=0)

    cluster_size = find_bighest_cluster_area(clusters_labels)

    return morphology.remove_small_objects(
        clusters,
        min_size=(cluster_size-1),
        connectivity=8
    )


def segmentation_roi(image):
    
    # Aplica K-means na imagem
    clusters = apply_kmeans(image, k_clusters=2)

    # Ecolhe melhor da segmentação Kmeans
    best_cluster = find_perfect_cluster(clusters)

    # Encontra maior cluster a mascara e remove as menores
    return find_best_larger_cluster((clusters == best_cluster))

    
def segmentation_mask_v2(image, background):
    
    image_eq = np.subtract(exposure.equalize_hist(image),
                           exposure.equalize_hist(background))
    
    image_roi = segmentation_roi(image_eq)
    
    crop_image = crop_image_box(image=image,
                                shape=find_center_mask(image_roi),
                                margin_pixel=80)
    # Detectando bordas
    canny_edges = feature.canny(exposure.equalize_hist(crop_image))
    sobel_edges = filters.sobel(canny_edges)

    # Conectando bordase preenchendo clusters
    closed_frame = morphology.closing(sobel_edges, morphology.disk(15))

    closed_frame = binarize_image(closed_frame)

    closed_frame = find_best_larger_cluster(closed_frame)

    return crop_image, ndi.binary_fill_holes(closed_frame)


In [16]:
def rule_of_three_percent_pixels(arr):
    
    """
        Essa função calcula a porcentagem de pixels Pretos e Brancos
        Essa informação é importante para selecionar imagens válidas para processamentos
    """
    
    def co_occurrence(arr):
        unique, counts = np.unique(arr, return_counts=True)
    
        return dict(zip(unique, counts))

    def ternary(value):
        return 0 if value == None else value
        
    def binarize_image(arr):
        return arr > filters.threshold_minimum(arr)

    image_bin = binarize_image(arr)
    image_coo = co_occurrence(image_bin)
    
    true_value = ternary(image_coo.get(True))
    false_value = ternary(image_coo.get(False)) 

    _100 = false_value + true_value
    
    return dict({
        'true_pixels': int((true_value * 100) / _100), 
        'false_pixels': int((false_value * 100) / _100)
    })


def check_colision_border(mask):
    """
        Pós processamento
    """
    x, *_ = mask.shape
    
    left = mask[:1,].flatten()
    right = mask[x - 1: x,].flatten()
    top = mask[:, : 1].flatten()
    bottom = mask[:, x - 1: x].flatten()
    
    borders_flatten = [left, right, top, bottom]
    if np.concatenate(borders_flatten).sum():
        return True
    
    return False

In [19]:
import tifffile
from glob import glob 
from tqdm import tqdm 
import os

def runner_v2(bg_image_path,
              img_name,
              path_out_folder,
              path_images_in_folder,
              fn_segmentation):

    path_images = f'{path_out_folder}/{img_name}/images'
    path_masks = f'{path_out_folder}/{img_name}/masks'
    
    try:
        os.makedirs(path_out_folder)
    except Exception: pass
        
    try:
        os.makedirs(path_images)
        os.makedirs(path_masks)
    except Exception: pass

    arr_images =  glob(path_images_in_folder + '/*')
    background = crop_image_reduce_errors(imread(bg_image_path)[:, :, 0])
    
    for index, frame_path in enumerate(tqdm(arr_images)):
        
        image_loaded = tifffile.imread(frame_path)[:, :, 0]
        image = crop_image_reduce_errors(image_loaded)

        try: percents = rule_of_three_percent_pixels(image) 
        except Exception: continue

        is_image_valid = percents['true_pixels'] > percents['false_pixels']

        # Ignora se a imagem for predominantemente preta
        if not is_image_valid: continue

        # Ignora qualquer error de segmentação e parte para o proximo frame
        try:
            image_cropped, mask = fn_segmentation(image, background)
        except Exception: continue

        # Ignora se a mascara colidir com as bordas
        if check_colision_border(mask): continue
        
        imsave(f'{path_images}/{str(index)}.tif', image_cropped)
        imsave(f'{path_masks}/{str(index)}.tif', img_as_ubyte(mask))

In [20]:
runner_v2(bg_image_path='background.tif',
          img_name='005',
          path_out_folder='results_outs',
          path_images_in_folder='dataset/images/005',
          fn_segmentation=segmentation_mask_v2)

100%|██████████| 257/257 [01:53<00:00,  2.25it/s]
